In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def vanderpol(lhs, mu):
    x1, x2 = lhs[0], lhs[1]
    rhs = np.zeros(2, dtype=np.float64)
    rhs[0] = x2
    rhs[1] = mu * (1. - x1 ** 2.) * x2 - x1
    return rhs

def duffing(lhs):
    """ Duffing oscillator:
    dx/dt = y
    dy/dt = x - x^3
    """
    x1, x2 = lhs[0], lhs[1]
    rhs = np.zeros(2)
    rhs[0] = x2
    rhs[1] = x1 - x1**3
    return rhs

def pendulum(lhs):
    rhs = np.zeros(2, dtype=np.float64)
    rhs[0] = lhs[1]
    rhs[1] = -np.sin(lhs[0])
    return rhs


def rk4(x0, f, dt):
    k1 = dt * f(x0)
    k2 = dt * f(x0 + k1 / 2.)
    k3 = dt * f(x0 + k2 / 2.)
    k4 = dt * f(x0 + k3)
    return x0 + (k1 + 2. * k2 + 2. * k3 + k4) / 6.

def timestepper(x0, t_start, t_stop, dt, f):
    ndim = np.size(x0)
    nsteps = np.int((t_stop - t_start) / dt)
    solpath = np.zeros((ndim, nsteps + 1), dtype=np.float64)
    solpath[:, 0] = x0
    for jj in range(nsteps):
        solpath[:, jj + 1] = rk4(solpath[:, jj], f, dt)
    return solpath

In [ ]:
def hankel_matrix(timeseries, window):
    num_time = len(timeseries)
    num_cols = num_time - (window - 1)
    hmat = np.zeros((window, num_cols), dtype=np.float64)
    for ii in range(num_cols):
        hmat[:, ii] = timeseries[ii:(ii + window)]
    sclfac = np.linalg.norm(hmat[:, -1])
    return [hmat, sclfac]


def create_big_hankel(x, window):
    num_ic, num_dims, num_time = x.shape
    print("num_time", num_time)
    num_rows = num_time - (window-1)
    print("num rows", num_rows)

    hankel_mats = np.zeros((num_ic, num_rows*num_dims, window), dtype=np.float64)
    for ii in range(num_ic):
        for jj in range(num_dims):
            timeseries = x[ii, jj, :]
            hmat, _ = hankel_matrix(timeseries, window)
            idx_start = jj*num_rows
            idx_stop = idx_start + num_rows
            hankel_mats[ii, idx_start:idx_stop, :] = hmat.T
    return hankel_mats

In [ ]:
def edmd(gtot, threshold, window):
    num_ic, num_rows, num_cols = gtot.shape
    evals = np.zeros((num_ic, num_rows), dtype=np.complex128)
    evecs = np.zeros((num_ic, num_rows, num_rows), dtype=np.complex128)
    phims = np.zeros((num_ic, num_rows, window-1), dtype=np.complex128)
    recon = np.zeros((num_ic, num_rows, window+100), dtype=np.float64)

    for ii in range(num_ic):
        gm = gtot[ii, :, :-1]
        gp = gtot[ii, :, 1:]

        u, s, vh = np.linalg.svd(gm, full_matrices=False)
        sm = np.max(s)
        indskp = np.log10(s / sm) > threshold
        sr = s[indskp]
        ur = u[:, indskp]
        v = np.conj(vh).T
        vr = v[:, indskp]

        kmat = gp @ vr @ np.diag(1.0/sr) @ np.conj(ur).T
        evals[ii, :], evecs[ii, :, :] = np.linalg.eig(kmat)
        phims[ii, :, :] = np.linalg.solve(evecs[ii, :, :], gm)

        g_init = phims[ii, :, 0]
        evals_tt = np.ones(np.size(g_init), dtype=np.complex128)
        for tt in range(window+100):
            recon[ii, :, tt] = np.real(evecs[ii, :, :] @ np.diag(evals_tt) @ g_init)
            evals_tt = evals_tt * evals[ii, :]

    return recon, evals, evecs, phims

In [ ]:
if __name__ == '__main__':
    # Setup
    np.random.seed(2021)
    dt = 0.05
    t0 = 0.0
    tf = 20.0
    window = 350 
    threshold = -10.0
    fhandle = lambda x: duffing(x)
    num_ic = 10
    dims = 2
    init_conds = np.zeros((num_ic, 2), dtype=np.float64)
    data = [None] * num_ic
    for ii in range(num_ic):
        init_conds[ii, :] = np.random.uniform(-1, 1, 2)
        data[ii] = timestepper(init_conds[ii, :], t0, tf, dt, fhandle)
    data = np.asarray(data)

    # Compute
    big_hankel = create_big_hankel(data, window)
    recon, evals, evecs, phims = edmd(big_hankel, threshold, window)

In [ ]:
plt.figure(11)
plt.scatter(np.real(evals), np.imag(evals))
plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), np.sin(np.linspace(0, 2*np.pi, 100)))

plt.figure(12)
ax = plt.subplot(2, 1, 1)
ax.set_title("Top=Data  Bottom=Recon")
for ii in range(data.shape[0]):
    ax.plot(data[ii, 0, :], data[ii, 1, :], '-')
ax.plot(data[:, 0, 0], data[:, 1, 0], 'o')
ax = plt.subplot(2, 1, 2)
num_time = data.shape[-1] - (window-1)
trajectories = recon[:, 0::num_time, :]
for ii in range(num_ic):
    x1 = trajectories[ii, 0, :]
    x2 = trajectories[ii, 1, :]
    ax.plot(x1, x2)
ax.plot(init_conds[:, 0], init_conds[:, 1], 'o')
# ax.set_xlim([-2, 2])
# ax.set_ylim([-1.5, 1.5])
plt.show()

print('done')